# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
csv_path = "../output_data/cities.csv"
cities_weather_df = pd.read_csv(csv_path)

cities_weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.1927,168.8643,70.48,46,86,5.21,NZ,1647664491
1,Ribeira,42.7461,-8.4439,51.73,85,0,3.44,ES,1647664491
2,Albany,42.6001,-73.9662,50.72,78,100,2.10,US,1647664249
3,Auki,-8.7676,160.7034,83.62,74,92,4.14,SB,1647664491
4,Nanlong,31.3533,106.0631,75.85,57,100,8.41,CN,1647664492
...,...,...,...,...,...,...,...,...,...
568,Bač,45.3919,19.2367,26.60,77,21,3.42,RS,1647664637
569,Seoul,37.5683,126.9778,42.37,81,100,13.80,KR,1647664402
570,Cervo,43.6702,-7.4101,50.22,80,9,5.12,ES,1647664637
571,Rajanpur,29.1035,70.3250,92.97,16,0,2.28,PK,1647664638


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key = g_key)

city_lat_lng = cities_weather_df[["Lat", "Lng"]]
humidity = cities_weather_df["Humidity"].astype(float)

figure_layout = {"width": "100%",
                 "height": "500px",
                 "padding": "1px"}

fig = gmaps.figure(layout = figure_layout, center = (30.0, 10.0), zoom_level = 2)
heat_layer = gmaps.heatmap_layer(city_lat_lng,
                                 weights = humidity,
                                 max_intensity = np.max(humidity),
                                 dissipating = False,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='500px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
ideal_locations_df = cities_weather_df.loc[(cities_weather_df["Max Temp"] >= 70) &
                                           (cities_weather_df["Max Temp"] <= 80) &
                                           (cities_weather_df["Wind Speed"] <= 10) &
                                           (cities_weather_df["Cloudiness"] == 0)].dropna()

ideal_locations_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,Macaé,-22.3708,-41.7869,74.95,100,0,4.61,BR,1647664453
87,Vallenar,-28.5708,-70.7581,71.49,39,0,1.03,CL,1647664515
258,Novo,22.7847,-82.2251,70.14,82,0,3.69,CU,1647664560
285,Nioro,13.3500,-15.7500,72.30,30,0,7.38,GM,1647664567
359,Santiago,-33.4569,-70.6483,74.17,39,0,3.44,CL,1647664584


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = ideal_locations_df.loc[:, ["City", "Lat", "Lng", "Country"]]
hotel_df["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"keyword": "hotel",
          "type": "lodging",
          "key": g_key,
          "radius": 5000,
          "location": ""}

for index, row in hotel_df.iterrows():
    
    params["location"] = str(row[1]) +","+ str(row[2])
    
    response = requests.get(base_url, params).json()
    
    hotel_results = response["results"]

    try:
        print(f"{row['City']}, {row['Country'].upper()} hotel: {hotel_results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = hotel_results[0]["name"]
    
    # Print error statement if hotels cannot be found
    except (KeyError, IndexError):
        print(f"There is no hotel found for {row['City']} within 5000 meters.")
    
hotel_df

Macaé, BR hotel: Royal Macaé Palace Hotel
Vallenar, CL hotel: Hotel Origenes
There is no hotel found for Novo within 5000 meters.
There is no hotel found for Nioro within 5000 meters.
Santiago, CL hotel: Sheraton Santiago Hotel and Convention Center
Acapulco de Juárez, MX hotel: Hotel Amarea Acapulco
Bosaso, SO hotel: Hotel Ramada


,City,Lat,Lng,Country,Hotel Name
32,Macaé,-22.3708,-41.7869,BR,Royal Macaé Palace Hotel
87,Vallenar,-28.5708,-70.7581,CL,Hotel Origenes
258,Novo,22.7847,-82.2251,CU,
285,Nioro,13.3500,-15.7500,GM,
359,Santiago,-33.4569,-70.6483,CL,Sheraton Santiago Hotel and Convention Center
423,Acapulco de Juárez,16.8634,-99.8901,MX,Hotel Amarea Acapulco
538,Bosaso,11.2842,49.1816,SO,Hotel Ramada


In [17]:
cities_count = hotel_df["City"].nunique()
print(f"There are %s cities that match the requirements" % cities_count)

There are 7 cities that match the requirements


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='500px', padding='1px', width='100%'))